In [129]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import csv
import re 

In [130]:
browser = webdriver.Edge(executable_path='driver\msedgedriver.exe')

# Maximize window
browser.maximize_window()

# Open google map
path_search = "https://www.google.com/maps/@9.779349,105.6189045,11z?hl=vi-VN"
browser.get(path_search)
sleep(2)

c:\Users\DELL\anaconda3\envs\vision37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [133]:
# Search 
input_search = browser.find_element(By.ID,"searchboxinput")
input_search.send_keys("nhà hàng quy nhơn")
input_search.send_keys(Keys.ENTER)
sleep(4)

In [134]:
# Get result search
output_searchs = browser.find_elements(By.XPATH, "//div[@class='Nv2PK THOPZb CpccDe']")
print(len(output_searchs))

5


In [135]:
def preprocess(text):
    html_pattern = re.compile('<.*?>')
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = html_pattern.sub(r' ', text)
    text = url_pattern.sub(r' ', text)
    text = text.replace("\n", "")
    text = text.replace(";", "")
    
    return text

In [136]:
for output_search in output_searchs:
    results = []
    
    # Choose each output
    output_search.click()
    sleep(3)
    
    # Get label review and click
    label_review = browser.find_element(By.XPATH, "//button[@class='DkEaL']")
    label_review.click()
    total_review = int(re.findall(r'\d+', label_review.text)[0])
    total_review = int(100*(total_review/100))
    print("scroll", total_review)
    sleep(3)
    
    # Scroll to get more result
    scroll = browser.find_element(By.XPATH, "//div[@class='m6QErb DxyBCb kA9KIf dS8AEf']")
    for _ in range(total_review):
        # Click "Xem thêm" to get full review
        try:
            see_more = browser.find_element(By.XPATH, "//button[@aria-label='Xem thêm']")
            see_more.click()
        except:
            pass
            
        # Scroll 
        sleep(0.5)
        scroll.send_keys(Keys.DOWN);
    
    # Get reviews
    reviews = browser.find_elements(By.XPATH, "//span[@class='wiI7pd']")
    print("total reviews", len(reviews))
    
    # Preproces review
    for review in reviews:
        text = review.text
        text = preprocess(text)
        if len(text) < 40:
            continue
        results.append([text])
    
    print("total results", len(results))
    
    # Write data to file csv
    with open(f"data_ggmap.csv", "a", encoding='utf-8', newline='') as file:
        # Create a CSV writer
        writer = csv.writer(file)
        
        # Write data
        writer.writerows(results)
    
    sleep(2)
# Đóng trình duyệt
browser.close()


scroll 40
total reviews 10
total results 9
scroll 366
total reviews 200
total results 169
scroll 167
total reviews 30
total results 15
scroll 566
total reviews 170
total results 145
scroll 405
total reviews 80
total results 75
